In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
# OPENAI_API_KEY

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
%pwd

In [ ]:
%cd ..

In [ ]:
%pwd

In [ ]:
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [ ]:
data

In [ ]:
len(data)

In [ ]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [ ]:
question_gen

In [ ]:
from langchain.text_splitter import TokenTextSplitter

In [ ]:
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 10000,
    chunk_overlap = 200
)

In [ ]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [ ]:
chunk_ques_gen

In [ ]:
len(chunk_ques_gen)

In [ ]:
type(chunk_ques_gen[0])

In [ ]:
from langchain.docstore.document import Document

In [ ]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [ ]:
document_ques_gen

In [ ]:
type(document_ques_gen[0])

In [ ]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [ ]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [ ]:
document_answer_gen

In [ ]:
len(document_answer_gen)

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [ ]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [ ]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)


In [ ]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [ ]:
from langchain.chains.summarize import load_summarize_chain

In [ ]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [ ]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [ ]:
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
ques

In [ ]:
ques_list = ques.split("\n")

In [ ]:
ques_list

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())


In [ ]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")


